In [1]:
import numpy as np
import pandas as pd
import calendar
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report

In [162]:
# preprocess=pd.read_csv('preprocess.csv', engine='python',index_col="arrival_date")
train=pd.read_csv('preprocess_all/preprocess_train_num_all.csv', engine='python')
# train=train.drop(["label"],axis=1)
train

,arrival_date,hotel_Resort,hotel_City,lead_time,arrival_date_week_number,stays_nights,adults,children,babies,meal_BB,...,wait_less_than_3,wait_between_3_30,wait_greater_than_30,contract,group,transient,transient_party,mean_required_car_parking_space,special_requests_ratio,quatity
0,2015/7/1,38,65,190.291262,27,3.048544,1.805825,0.019417,0.000000,35,...,103,0,0,6,0,97,0,0.038835,0.184466,103
1,2015/7/2,35,1,69.944444,27,5.694444,1.972222,0.055556,0.000000,27,...,36,0,0,13,0,20,3,0.138889,0.500000,36
2,2015/7/3,27,10,65.378378,27,4.324324,2.000000,0.081081,0.000000,22,...,37,0,0,6,0,22,9,0.189189,0.243243,37
3,2015/7/4,36,9,120.777778,27,5.311111,1.977778,0.111111,0.044444,37,...,45,0,0,7,0,26,12,0.155556,0.488889,45
4,2015/7/5,37,0,75.378378,28,6.108108,1.945946,0.216216,0.000000,26,...,37,0,0,9,1,27,0,0.351351,0.378378,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,2017/3/27,25,89,96.368421,13,2.982456,1.842105,0.000000,0.000000,99,...,114,0,0,0,2,67,45,0.070175,0.675439,114
636,2017/3/28,33,34,52.537313,13,3.820896,1.791045,0.044776,0.014925,54,...,67,0,0,0,1,64,2,0.059701,0.492537,67
637,2017/3/29,51,79,41.130769,13,2.800000,1.653846,0.053846,0.015385,104,...,130,0,0,0,3,125,2,0.100000,0.484615,130
638,2017/3/30,51,73,59.137097,13,3.596774,1.725806,0.040323,0.024194,85,...,124,0,0,3,0,102,19,0.072581,0.508065,124


In [52]:
y_label=pd.read_csv('train_label.csv', engine='python',index_col="arrival_date")
# test_nolabel = pd.read_csv('test_nolabel.csv', engine='python')
y_label_tr=pd.get_dummies(y_label, columns=["label"])
y_label_tr

,label_0.0,label_1.0,label_2.0,label_3.0,label_4.0,label_5.0,label_6.0,label_7.0,label_8.0,label_9.0
arrival_date,,,,,,,,,,
2015-07-01,0,0,1,0,0,0,0,0,0,0
2015-07-02,0,1,0,0,0,0,0,0,0,0
2015-07-03,0,1,0,0,0,0,0,0,0,0
2015-07-04,0,1,0,0,0,0,0,0,0,0
2015-07-05,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2017-03-27,0,0,1,0,0,0,0,0,0,0
2017-03-28,0,1,0,0,0,0,0,0,0,0
2017-03-29,0,0,1,0,0,0,0,0,0,0


In [21]:
def augFeatures(train):
    train["arrival_date"] = pd.to_datetime(train["arrival_date"])
    train["year"] = train["arrival_date"].dt.year
    train["month"] = train["arrival_date"].dt.month
    train["date"] = train["arrival_date"].dt.day
    train["day"] = train["arrival_date"].dt.dayofweek
    train=train.drop(["arrival_date"],axis=1)
    return train


In [7]:
def normalize(train):
    train = train.drop(["Date"], axis=1)
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm

In [78]:
def buildTrain(train,y_label, pastDay, futureDay):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
#         Y_train.append(np.array(y_label.iloc[i+pastDay:i+pastDay+futureDay]))
        yy=np.array(y_label.iloc[i+pastDay:i+pastDay+futureDay])
        Y_train.append(np.reshape(yy, (yy.shape[1])))
#     a=np.array(y_label.iloc[i+pastDay:i+pastDay+futureDay])
#     print(np.reshape(a, (a.shape[1])).shape)
#     print(np.array(Y_train).shape)
    return np.array(X_train), np.array(Y_train)


In [37]:
def shuffle(X,Y):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

In [38]:
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

In [156]:
def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(128, input_length=shape[1], input_dim=shape[2],return_sequences=True))
#     model.add(LSTM(units = 256, input_dim=42,return_sequences=True))
    model.add(LSTM(64))#new
#     model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    # output shape: (1, 1)
    model.add(Dense(10, activation='softmax'))
    model.compile(loss="binary_crossentropy", optimizer="adam")
#     model.compile(loss="categorical_crossentropy", optimizer="adam")
    model.summary()
    return model

In [108]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM,Embedding
from keras import optimizers
from keras.callbacks import EarlyStopping

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
# train = readTrain()
# train_Aug = augFeatures(train)
train_Aug=train
train_Aug=train_Aug.drop(["arrival_date"],axis=1)
print(train_Aug.shape)
# train_norm = normalize(train_Aug)
train_norm = train_Aug
print(train_norm.shape)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm,y_label_tr, 2, 1)
print(Y_train.shape)
X_train, Y_train = shuffle(X_train, Y_train)
print(Y_train.shape)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.2)

model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=200, validation_data=(X_val, Y_val), callbacks=[callback])
y_pred = model.predict(X_val, batch_size=200, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
y_test_bool = np.argmax(np.array(Y_val), axis=1)
print(classification_report(y_test_bool, y_pred_bool))
print(confusion_matrix(y_test_bool, y_pred_bool))
print(y_pred)

In [64]:
Y_val.shape

(125, 1, 10)

In [50]:
import numpy as np
train_lstm=np.array(train)
X_train_all = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
y_label_tr=pd.get_dummies(y_label, columns=["label"])

In [48]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM,Embedding
from keras import optimizers
from keras.callbacks import EarlyStopping

In [68]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [152]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
kfold=KFold(5,True,0)
cvscores = []
cvscores_f1 = []
for train1, test1 in kfold.split(train,y_label):
#     print(train1)
    X_train=np.array(train.iloc[train1])
    X_test=np.array(train.iloc[test1])
    y_train=y_label_tr.iloc[train1]
    y_test=y_label_tr.iloc[test1]
#     X_train, X_test, y_train, y_test = train_test_split(train_lstm, y_label_tr, test_size = 0.2, random_state = 0)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test,(X_test.shape[0], 1, X_test.shape[1]))
#     print(y_train)

    model = Sequential()

#     model.add(LSTM(units = 256, input_dim=42))
    model.add(LSTM(units = 256, input_dim=42,return_sequences=True))
    model.add(LSTM(128,return_sequences=True))#new
    model.add(LSTM(64))#new
    model.add(Dropout(0.2))
    model.add(Dense(32))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
    model.add(Dense(10, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['mae',f1_m,'accuracy'])
    model.fit(X_train, y_train, batch_size=20, epochs=40, verbose=0)
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)
    

    y_pred = model.predict(X_test, batch_size=64, verbose=0)
    y_pred_bool = np.argmax(y_pred, axis=1)
    y_test_bool = np.argmax(np.array(y_test), axis=1)
    print(classification_report(y_test_bool, y_pred_bool))
    print(confusion_matrix(y_test_bool, y_pred_bool))
# print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

NameError: name 'f1_m' is not defined

In [154]:
from sklearn.model_selection import train_test_split
# train2=train.drop(["arrival_date"],axis=1)
train2 = augFeatures(train)
X_train, X_test, y_train, y_test = train_test_split(train2, y_label_tr, test_size=0.2, random_state=42)
X_train=np.array(X_train)
X_test=np.array(X_test)
# y_train=y_label_tr.iloc[train1]
# y_test=y_label_tr.iloc[test1]
#     X_train, X_test, y_train, y_test = train_test_split(train_lstm, y_label_tr, test_size = 0.2, random_state = 0)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test,(X_test.shape[0], 1, X_test.shape[1]))
#     print(y_train)

model = Sequential()

#     model.add(LSTM(units = 256, input_dim=42))
# model.add(LSTM(units = 128, input_dim=42,return_sequences=True))
model.add(LSTM(units = 256, input_dim=46,return_sequences=True))
model.add(LSTM(128,return_sequences=True))#new
model.add(LSTM(64))#new
# model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam')
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['mae',f1_m,'accuracy'])
# model.fit(X_train, y_train, batch_size=20, epochs=40, verbose=0)
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.fit(X_train, y_train, epochs=1000, batch_size=200, validation_data=(X_test, y_test), callbacks=[callback])
y_pred = model.predict(X_test, batch_size=200, verbose=0)  

# y_pred = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
y_test_bool = np.argmax(np.array(y_test), axis=1)
print(classification_report(y_test_bool, y_pred_bool))
print(confusion_matrix(y_test_bool, y_pred_bool))

Train on 512 samples, validate on 128 samples
Epoch 1/1000
512/512 [==============================] - 6s 12ms/step - loss: 0.6892 - val_loss: 0.6787
Epoch 2/1000
512/512 [==============================] - 0s 96us/step - loss: 0.6733 - val_loss: 0.6573
Epoch 3/1000
512/512 [==============================] - 0s 94us/step - loss: 0.6495 - val_loss: 0.6273
Epoch 4/1000
512/512 [==============================] - 0s 91us/step - loss: 0.6168 - val_loss: 0.5878
Epoch 5/1000
512/512 [==============================] - 0s 90us/step - loss: 0.5746 - val_loss: 0.5385
Epoch 6/1000
512/512 [==============================] - 0s 98us/step - loss: 0.5228 - val_loss: 0.4831
Epoch 7/1000
512/512 [==============================] - 0s 101us/step - loss: 0.4672 - val_loss: 0.4295
Epoch 8/1000
512/512 [==============================] - 0s 101us/step - loss: 0.4155 - val_loss: 0.3847
Epoch 9/1000
512/512 [==============================] - 0s 100us/step - loss: 0.3738 - val_loss: 0.3511
Epoch 10/1000
512/512 [=

In [166]:
train_lstm

array([['2015/7/1', 38, 65, ..., 0.038834951, 0.18446601899999998, 103],
       ['2015/7/2', 35, 1, ..., 0.13888888900000002, 0.5, 36],
       ['2015/7/3', 27, 10, ..., 0.18918918899999998, 0.243243243, 37],
       ...,
       ['2017/3/29', 51, 79, ..., 0.1, 0.484615385, 130],
       ['2017/3/30', 51, 73, ..., 0.072580645, 0.508064516, 124],
       ['2017/3/31', 64, 76, ..., 0.061538462, 0.592307692, 130]],
      dtype=object)

In [176]:
# train_lstm=np.array(train)
train_lstm=train.drop(["arrival_date"],axis=1)
train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
# X_train, X_test, y_train, y_test = train_test_split(train2, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 256, input_dim=42,return_sequences=True))
model.add(LSTM(128,return_sequences=True))#new
model.add(LSTM(64))#new
# model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_label_tr, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_train, batch_size=64, verbose=0)
y_pred_bool_train = np.argmax(y_pred_train, axis=1)
y_train_bool = np.argmax(np.array(y_label_tr), axis=1)
print(classification_report(y_train_bool, y_pred_bool_train))
print(confusion_matrix(y_train_bool, y_pred_bool_train))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        94
           1       1.00      1.00      1.00       152
           2       0.99      0.99      0.99       186
           3       1.00      0.99      1.00       124
           4       0.96      0.98      0.97        46
           5       0.95      1.00      0.98        21
           6       0.87      1.00      0.93        13
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
           9       1.00      1.00      1.00         1

    accuracy                           0.99       640
   macro avg       0.78      0.80      0.79       640
weighted avg       0.99      0.99      0.99       640

[[ 94   0   0   0   0   0   0   0   0   0]
 [  0 152   0   0   0   0   0   0   0   0]
 [  0   0 185   0   1   0   0   0   0   0]
 [  0   0   0 123   1   0   0   0   0   0]
 [  0   0   1   0  45   0   0   0   0   0]
 [  0   0   0   0   0  21 

In [177]:
error = []
for i in range(len(y_train_bool)):
    error.append(y_train_bool[i] - y_pred_bool_train[i])

# print("Errors: ", error)
# print(error)

absError = []
for val in error:
    absError.append(abs(val))#誤差絕對值

# print("Absolute Value of Error: ", absError)
from math import sqrt
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE
 

MAE =  0.0171875


In [179]:
# train_lstm=np.array(train)
train_lstm=train.drop(["arrival_date"],axis=1)
train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 256, input_dim=42,return_sequences=True))
model.add(LSTM(128,return_sequences=True))#new
model.add(LSTM(64))#new
# model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_train, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_train, axis=1)
y_test_bool = np.argmax(np.array(y_test), axis=1)
print(classification_report(y_test_bool, y_pred_bool_test))
print(confusion_matrix(y_test_bool, y_pred_bool_test))

              precision    recall  f1-score   support

           0       0.64      0.80      0.71        20
           1       0.64      0.68      0.66        31
           2       0.60      0.44      0.51        34
           3       0.45      0.52      0.48        27
           4       0.33      0.40      0.36        10
           5       0.50      0.33      0.40         3
           6       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy                           0.55       128
   macro avg       0.40      0.40      0.39       128
weighted avg       0.55      0.55      0.54       128

[[16  3  1  0  0  0  0  0]
 [ 7 21  3  0  0  0  0  0]
 [ 0  8 15  9  2  0  0  0]
 [ 2  1  5 14  5  0  0  0]
 [ 0  0  0  6  4  0  0  0]
 [ 0  0  0  1  1  1  0  0]
 [ 0  0  0  1  0  1  0  0]
 [ 0  0  1  0  0  0  0  0]]


In [181]:
error = []
for i in range(len(y_test_bool)):
    error.append(y_test_bool[i] - y_pred_bool_test[i])

# print("Errors: ", error)
# print(error)

absError = []
for val in error:
    absError.append(abs(val))#誤差絕對值

# print("Absolute Value of Error: ", absError)
from math import sqrt
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE
 

MAE =  0.578125


In [183]:
# train_lstm=np.array(train)
train_lstm=train.drop(["arrival_date"],axis=1)
train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 256, input_dim=42,return_sequences=True))
model.add(LSTM(128))#new
# model.add(LSTM(64))#new
# model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_train, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_train, axis=1)
y_test_bool = np.argmax(np.array(y_test), axis=1)
print(classification_report(y_test_bool, y_pred_bool_test))
print(confusion_matrix(y_test_bool, y_pred_bool_test))

              precision    recall  f1-score   support

           0       0.74      0.85      0.79        20
           1       0.71      0.77      0.74        31
           2       0.58      0.53      0.55        34
           3       0.33      0.33      0.33        27
           4       0.38      0.30      0.33        10
           5       0.20      0.33      0.25         3
           6       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy                           0.56       128
   macro avg       0.37      0.39      0.37       128
weighted avg       0.54      0.56      0.55       128

[[17  3  0  0  0  0  0  0]
 [ 4 24  3  0  0  0  0  0]
 [ 0  6 18 10  0  0  0  0]
 [ 2  1  9  9  3  3  0  0]
 [ 0  0  1  6  3  0  0  0]
 [ 0  0  0  1  1  1  0  0]
 [ 0  0  0  0  1  1  0  0]
 [ 0  0  0  1  0  0  0  0]]


In [184]:
error = []
for i in range(len(y_test_bool)):
    error.append(y_test_bool[i] - y_pred_bool_test[i])

# print("Errors: ", error)
# print(error)

absError = []
for val in error:
    absError.append(abs(val))#誤差絕對值

# print("Absolute Value of Error: ", absError)
from math import sqrt
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE
 

MAE =  0.5625


In [185]:
# train_lstm=np.array(train)
train_lstm=train.drop(["arrival_date"],axis=1)
train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 256, input_dim=42,return_sequences=True))
model.add(LSTM(128))#new
# model.add(LSTM(64))#new
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_train, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_train, axis=1)
y_test_bool = np.argmax(np.array(y_test), axis=1)
print(classification_report(y_test_bool, y_pred_bool_test))
print(confusion_matrix(y_test_bool, y_pred_bool_test))

              precision    recall  f1-score   support

           0       0.70      0.80      0.74        20
           1       0.68      0.68      0.68        31
           2       0.56      0.65      0.60        34
           3       0.52      0.48      0.50        27
           4       0.43      0.30      0.35        10
           5       0.33      0.33      0.33         3
           6       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy                           0.59       128
   macro avg       0.40      0.40      0.40       128
weighted avg       0.57      0.59      0.58       128

[[16  3  1  0  0  0  0  0]
 [ 6 21  4  0  0  0  0  0]
 [ 0  7 22  5  0  0  0  0]
 [ 1  0 10 13  2  1  0  0]
 [ 0  0  1  6  3  0  0  0]
 [ 0  0  0  1  1  1  0  0]
 [ 0  0  0  0  1  1  0  0]
 [ 0  0  1  0  0  0  0  0]]


In [186]:
error = []
for i in range(len(y_test_bool)):
    error.append(y_test_bool[i] - y_pred_bool_test[i])

# print("Errors: ", error)
# print(error)

absError = []
for val in error:
    absError.append(abs(val))#誤差絕對值

# print("Absolute Value of Error: ", absError)
from math import sqrt
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE
 

MAE =  0.5078125


In [187]:
# train_lstm=np.array(train)
train_lstm=train.drop(["arrival_date"],axis=1)
train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 256, input_dim=42))
# model.add(LSTM(128))#new
# model.add(LSTM(64))#new
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_train, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_train, axis=1)
y_test_bool = np.argmax(np.array(y_test), axis=1)
print(classification_report(y_test_bool, y_pred_bool_test))
print(confusion_matrix(y_test_bool, y_pred_bool_test))

              precision    recall  f1-score   support

           0       0.71      0.85      0.77        20
           1       0.67      0.65      0.66        31
           2       0.49      0.56      0.52        34
           3       0.48      0.48      0.48        27
           4       0.43      0.30      0.35        10
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy                           0.56       128
   macro avg       0.35      0.35      0.35       128
weighted avg       0.54      0.56      0.55       128

[[17  1  2  0  0  0  0  0]
 [ 6 20  5  0  0  0  0  0]
 [ 0  8 19  7  0  0  0  0]
 [ 1  0 10 13  3  0  0  0]
 [ 0  1  1  5  3  0  0  0]
 [ 0  0  1  1  1  0  0  0]
 [ 0  0  0  1  0  1  0  0]
 [ 0  0  1  0  0  0  0  0]]


In [188]:
error = []
for i in range(len(y_test_bool)):
    error.append(y_test_bool[i] - y_pred_bool_test[i])

# print("Errors: ", error)
# print(error)

absError = []
for val in error:
    absError.append(abs(val))#誤差絕對值

# print("Absolute Value of Error: ", absError)
from math import sqrt
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE
 

MAE =  0.578125


In [189]:
# train_lstm=np.array(train)
train_lstm=train.drop(["arrival_date"],axis=1)
train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 256, input_dim=42))
# model.add(LSTM(128))#new
# model.add(LSTM(64))#new
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_train, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_train, axis=1)
y_test_bool = np.argmax(np.array(y_test), axis=1)
print(classification_report(y_test_bool, y_pred_bool_test))
print(confusion_matrix(y_test_bool, y_pred_bool_test))

              precision    recall  f1-score   support

           0       0.84      0.80      0.82        20
           1       0.69      0.87      0.77        31
           2       0.56      0.53      0.55        34
           3       0.54      0.56      0.55        27
           4       0.33      0.30      0.32        10
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy                           0.62       128
   macro avg       0.37      0.38      0.37       128
weighted avg       0.59      0.62      0.60       128

[[16  2  2  0  0  0  0  0]
 [ 2 27  2  0  0  0  0  0]
 [ 0  8 18  7  1  0  0  0]
 [ 1  2  7 15  2  0  0  0]
 [ 0  0  2  5  3  0  0  0]
 [ 0  0  0  1  2  0  0  0]
 [ 0  0  0  0  1  1  0  0]
 [ 0  0  1  0  0  0  0  0]]


In [190]:
error = []
for i in range(len(y_test_bool)):
    error.append(y_test_bool[i] - y_pred_bool_test[i])

# print("Errors: ", error)
# print(error)

absError = []
for val in error:
    absError.append(abs(val))#誤差絕對值

# print("Absolute Value of Error: ", absError)
from math import sqrt
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE
 

MAE =  0.515625


In [194]:
# train_lstm=np.array(train)
train_lstm=train.drop(["arrival_date"],axis=1)
train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 256, input_dim=42))
# model.add(LSTM(128))#new
# model.add(LSTM(64))#new
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.25))
# model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_train, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_train, axis=1)
y_test_bool = np.argmax(np.array(y_test), axis=1)
print(classification_report(y_test_bool, y_pred_bool_test))
print(confusion_matrix(y_test_bool, y_pred_bool_test))

              precision    recall  f1-score   support

           0       0.71      0.75      0.73        20
           1       0.65      0.71      0.68        31
           2       0.53      0.56      0.54        34
           3       0.46      0.41      0.43        27
           4       0.40      0.40      0.40        10
           5       0.33      0.33      0.33         3
           6       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy                           0.56       128
   macro avg       0.39      0.39      0.39       128
weighted avg       0.54      0.56      0.55       128

[[15  3  2  0  0  0  0  0]
 [ 4 22  5  0  0  0  0  0]
 [ 0  7 19  5  3  0  0  0]
 [ 2  1 10 11  2  1  0  0]
 [ 0  0  0  6  4  0  0  0]
 [ 0  0  0  2  0  1  0  0]
 [ 0  0  0  0  1  1  0  0]
 [ 0  1  0  0  0  0  0  0]]


In [195]:
error = []
for i in range(len(y_test_bool)):
    error.append(y_test_bool[i] - y_pred_bool_test[i])

# print("Errors: ", error)
# print(error)

absError = []
for val in error:
    absError.append(abs(val))#誤差絕對值

# print("Absolute Value of Error: ", absError)
from math import sqrt
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE
 

MAE =  0.6015625


In [196]:
# train_lstm=np.array(train)
train_lstm=train.drop(["arrival_date"],axis=1)
train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 128, input_dim=42))
# model.add(LSTM(128))#new
# model.add(LSTM(64))#new
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_train, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_train, axis=1)
y_test_bool = np.argmax(np.array(y_test), axis=1)
print(classification_report(y_test_bool, y_pred_bool_test))
print(confusion_matrix(y_test_bool, y_pred_bool_test))

              precision    recall  f1-score   support

           0       0.65      0.75      0.70        20
           1       0.59      0.65      0.62        31
           2       0.54      0.41      0.47        34
           3       0.45      0.52      0.48        27
           4       0.30      0.30      0.30        10
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy                           0.52       128
   macro avg       0.32      0.33      0.32       128
weighted avg       0.51      0.52      0.51       128

[[15  3  1  1  0  0  0  0]
 [ 6 20  5  0  0  0  0  0]
 [ 1  8 14  9  2  0  0  0]
 [ 1  3  3 14  4  2  0  0]
 [ 0  0  2  4  3  1  0  0]
 [ 0  0  0  2  1  0  0  0]
 [ 0  0  0  1  0  1  0  0]
 [ 0  0  1  0  0  0  0  0]]


In [197]:
error = []
for i in range(len(y_test_bool)):
    error.append(y_test_bool[i] - y_pred_bool_test[i])

# print("Errors: ", error)
# print(error)

absError = []
for val in error:
    absError.append(abs(val))#誤差絕對值

# print("Absolute Value of Error: ", absError)
from math import sqrt
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE
 

MAE =  0.6796875
